In [1]:
!pip install langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core duckduckgo-search langchain-community chromadb


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import dotenv
dotenv.load_dotenv()

True

# 커스텀 툴 정의하기

에이전트를 구성할 때, 에이전트가 사용할 수 있는 도구의 목록(Tools)을 제공해야함.

도구는 호출되는 실제 함수 외에도 여러 구성 요소로 이루어져 있음.

| Attribute  | Type  |  Description |
|---|---|---|
| name  | str  | LLM 또는 에이전트에 제공된 도구 세트 내에서 고유(unique)해야 합니다.  |
| description | str  |  도구가 무엇을 하는지 설명합니다. LLM 또는 에이전트에서 문맥으로 사용됩니다. |
| args_schema	| pydantic.BaseModel  | 선택 사항이지만 권장되며, 콜백 핸들러를 사용하는 경우 필수입니다. 예상되는 매개변수에 대한 추가 정보(예: few-shot examples) 또는 검증을 제공하는 데 사용할 수 있습니다. |
| return_direct		| boolean  |  에이전트에만 관련됩니다. True로 설정된 경우, 지정된 도구를 호출한 후 에이전트가 작업을 중지하고 결과를 사용자에게 직접 반환합니다. |


# Creating tools from functions

## @tool decorator
@tool 데코레이터는 사용자 정의 도구(custom tool)를 정의하는 가장 간단한 방법입니다. 이 데코레이터는 기본적으로 함수 이름을 도구 이름으로 사용하지만,
첫 번째 인자로 문자열을 전달하여 이를 재정의할 수 있습니다.
또한, 함수의 docstring 을 도구의 설명으로 사용하므로 반드시 docstring을 제공해야합니다.

In [2]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


@tool 데코레이터는 주석(annotation) parsing, 중첩 스키마(nested schemas), 기타 기능을 지원

In [3]:
from typing import Annotated, List

@tool
def multiply_by_max(
    a: Annotated[int, "scale factor"],
    b: Annotated[List[int], "List of ints over which to take maximum"],
) -> int:
    """Multipy a by the maximum of b."""
    return a * max(b)

print(multiply_by_max.args_schema.model_json_schema())

{'description': 'Multipy a by the maximum of b.', 'properties': {'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'List of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}, 'required': ['a', 'b'], 'title': 'multiply_by_max', 'type': 'object'}


## Docstring Parsing

@tool은 선택적으로 Google 스타일 닥스트링( https://google.github.io/styleguide/pyguide.html#383-functions-and-methods )을 분석하고 닥스트링 구성 요소(예: 인자 설명)를 도구 스키마의 관련 부분과 연결할 수 있습니다. 이 동작을 활성화하려면 parse_docstring을 지정

주의(caution) : 기본적으로 @tool(parse_docstring=True)는 닥스트링이 올바르게 분석되지 않을 경우 ValueError를 발생

In [4]:
@tool(parse_docstring=True)
def foo(bar: str, baz: int) -> str:
    """The foo.

    Args:
        bar: The bar.
        baz: The baz.
    """
    return bar


print(foo.args_schema.model_json_schema())

{'description': 'The foo.', 'properties': {'bar': {'description': 'The bar.', 'title': 'Bar', 'type': 'string'}, 'baz': {'description': 'The baz.', 'title': 'Baz', 'type': 'integer'}}, 'required': ['bar', 'baz'], 'title': 'foo', 'type': 'object'}


# LLM과 정의한 Custom Tool 연동

In [6]:
tools = [multiply]

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain import hub

# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant.

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [9]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=runnable_agent, tools=tools, handle_parsing_errors=True
)

In [10]:
results = agent_executor.invoke({"input":"6*7을 계산해줘"})

In [11]:
results

{'input': '6*7을 계산해줘', 'output': '6 * 7의 결과는 42입니다.'}

In [12]:
results = agent_executor.invoke({"input":"황제 펭귄에 대해 알려줘"})
results

{'input': '황제 펭귄에 대해 알려줘',
 'output': '황제 펭귄(Emperor Penguin)은 펭귄의 일종으로, 가장 큰 펭귄 종입니다. 다음은 황제 펭귄에 대한 몇 가지 주요 정보입니다:\n\n1. **서식지**: 황제 펭귄은 주로 남극 대륙의 해안과 주변 섬에서 서식합니다. 극한의 추위와 얼음 환경에서 살아가는 능력이 뛰어납니다.\n\n2. **크기**: 성체 황제 펭귄은 약 1.2미터에서 1.3미터 정도 자라며, 체중은 22kg에서 45kg까지 다양합니다. 수컷과 암컷의 크기는 비슷하지만, 수컷이 약간 더 큽니다.\n\n3. **번식**: 황제 펭귄은 독특한 번식 방식을 가지고 있습니다. 겨울철에 번식하며, 암컷이 알을 낳으면 수컷이 알을 품고, 암컷은 먹이를 찾으러 나갑니다. 수컷은 알을 발바닥에 두고 따뜻하게 유지하며, 약 2개월 동안 먹지 않고 지킵니다.\n\n4. **먹이**: 황제 펭귄은 주로 물고기, 오징어, 크릴 등을 먹습니다. 수중에서 뛰어난 수영 능력을 발휘하여 먹이를 잡습니다.\n\n5. **사회적 행동**: 황제 펭귄은 사회적 동물로, 큰 무리를 이루어 생활합니다. 서로의 체온을 유지하기 위해 밀집하여 서 있는 행동이 특징적입니다.\n\n6. **보전 상태**: 황제 펭귄은 기후 변화와 해빙 감소로 인해 서식지가 위협받고 있으며, 이로 인해 보전 상태가 우려되고 있습니다.\n\n황제 펭귄은 그 독특한 생태와 생존 방식으로 많은 사람들에게 사랑받고 있는 동물입니다.'}